# Circuit-QED

This notebook contains some useful models for superconducting circuit setups, such as charge qubits, flux qubits and combinations thereof.

In [ ]:
# file: seeq/models/cqed.py
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import LinearOperator
from seeq.tools import lowest_eigenstates, lowest_eigenvalues
import copy

## Transmon qubit

### Effective model

A charge qubit is a superconductiong circuit made of a capacitor and a Josephson junction (See Fig. a) below). The transmon is a variant of the charge qubit where the Josephson energy is much larger than the capacitive energy. This change makes the transmon less sensitive to charge and voltage fluctuations.

![Charge and transmon qubits, and equivalent circuit](figures/transmon.png)

Following the image above, the Hamiltonian for the transmon qubit is
$$\hat{H}=\frac{1}{2C_\Sigma} (\hat{q}-q_g)^2-E_J \cos(\hat{\varphi})$$
where $q_g=-C_g V$ is the equilibrium charge, $C_g$ is the control capacitance, $V$ the electric potential of the battery, $E_J$ the Josephson junction energy, and $C_\Sigma$ the combined qubit and charging capacitance. $\hat{q}$ is the charge operator of the Cooper pairs, and $\hat{\varphi}$ the flux operator.

We work with this Hamiltonian by introducing the charge number states
$$\hat{q} \lvert n \rangle = -2e n \lvert n \rangle, \quad \text{and} \quad \langle \varphi \lvert n \rangle \sim \frac{1}{\sqrt{2\pi}} e^{-in\varphi}$$

Using the number basis we can represent $\cos(\hat{\varphi})$ as
$$\cos(\hat{\varphi})=\frac{1}{2}\sum_{n\in\mathbb{Z}} \lvert n+1\rangle \langle n\rvert + \vert n \rangle \langle n+1 \rvert$$

Then we can express the charge qubit Hamiltonian in the number representation as
$$ H=\sum_{n\in\mathbb{Z}} \left[ 4 E_C (n-n_g)^2 \lvert n\rangle \langle n\rvert - \frac{E_J}{2}(\lvert n+1\rangle \langle n\rvert + \vert n \rangle \langle n+1 \rvert)\right]$$
with the charging energy $E_C=e^2/2C_\Sigma$.

This operator is an infinite-dimensional matrix
$$H = \left(\begin{matrix}
\ldots & -E_J/2 & 4E_c(-1-n_g)^2 & -E_J/2 & 0 & \ldots\\
\ldots & 0 & -E_J/2 & 4E_c(0-n_g)^2 & -E_J/2 & \ldots\\
\ldots & 0 & 0 & -E_J/2 & 4E_c(+1-n_g)^2 & \ldots
\end{matrix}\right)$$
but it can be truncated to a work in a limit $-n_{max}\leq n \leq n_{max},$ considering enough charge states as to provide a good approximation. In this case, we can write the model as
$$H = 4 E_C (\bar{N}-n_g)^2 - \frac{1}{2}E_J (\bar{S}^+ + \bar{S}^-),$$
with finite dimensional matrices $\bar{N}$ for the number of charges, and charge raising and lowering operators $\bar{S}^\pm.$

We can also introduce a model in which we have $M$ transmons interacting with each other capacitively. If we neglect the renormalization of the transmon capacitance, the effective model reads
$$H = \sum_i \left[4 E_{C,i} (\bar{N}_i-n_{g,i})^2 + \frac{1}{2}E_{J,i}(\bar{S}^+_i + \bar{S}^-_i)\right] + \sum_{i\neq j} g_{ij} \bar{N}_i\bar{N}_j.$$

In [ ]:
# file: seeq/models/cqed.py

class Transmons(LinearOperator):
    
    """Transmons() implements one or more coupled transmons. This class
    acts as a LinearOperator that implements the Hamiltonian. It can
    also produce copies of itself with adjusted parameters. If a parameter
    is a scalar, the same value is used for all qubits.
    
    Parameters
    ----------
    nqubits -- number of transmons
    Ec      -- capacitive energy (defaults to 1/95.)
    EJ      -- Josephson energy (defaults to 1.).
    g       -- couplings (scalar or matrix)
    ng      -- offset in number (defaults to 0)
    nmax    -- cutoff in charge space (defaults to 8)
    format  -- format of matrices (defaults to 'csr')
    """

    def __init__(self, nqubits, Ec=1/95., EJ=1., g=0, ng=0, nmax=8, format='csr'):
        self.nqubits = nqubits
        self.Ec = Ec = Ec * np.ones(nqubits)
        self.ng = ng = ng * np.ones(nqubits)
        self.EJ = EJ = EJ * np.ones(nqubits)
        assert len(Ec) == len(ng) == len(EJ) == nqubits
        # Dimensions of one-qubit problem
        dim = 2*nmax+1
        # Dimension of operators and states for the full problem
        fulldim = dim**nqubits
        #
        # This class inherits from LinearOperator because that implements
        # useful multiplication operators.
        super(Transmons,self).__init__(np.float64, (fulldim,fulldim))       
        #
        # Operators for one qubit
        self.nmax = nmax
        N = sp.diags(np.arange(-nmax, nmax + 1, 1), 0,
                     shape=(dim, dim), format=format)
        Sup = sp.diags([1.0], [1], shape=(dim,dim), format=format)
        Sdo = Sup.T
        #
        # Extend an operator to act on the whole Hilbert space
        def qubit_operator(op, j, N):
            d = op.shape[0]
            il = sp.eye(d**j, format=format)
            ir = sp.eye(d**(N-j-1), format=format)
            return sp.kron(il, sp.kron(op, ir))
        #
        # Local operators on all qubits:
        #
        self.N = [qubit_operator(N, j, nqubits) for j in range(nqubits)]
        self.nmax = nmax
        #
        # Capacitive energy
        Id = sp.eye(fulldim)
        self.Hcap = sum((4.0*Ec) * (N-ng*Id)**2
                        for ng, Ec, N in zip(ng, self.Ec, self.N))
        #
        # Inductive energy
        self.HJJ = [qubit_operator((Sup+Sdo)/2., j, nqubits)
                    for j, EJ in enumerate(self.EJ)]
        #
        # The interaction must be symmetric
        g = g * np.ones((nqubits, nqubits))
        self.g = (g + g.T)/2.0

    def hamiltonian(self):
        """Return the Hamiltonian of this set of transmons."""
        return self.Hcap + \
            sum((-EJ) * hi for EJ, hi in zip(self.EJ,self.HJJ)) + \
            sum((2*self.g[i,j]) * (self.N[i] * self.N[j])
                     for i in range(self.nqubits)
                     for j in range(i)
                     if self.g[i,j])
            
    def apply(self, ψ):
        """Act with the Hamiltonian of this set of transmons, onto
        the state ψ."""
        g = self.g
        N = self.N
        return self.Hcap @ ψ \
            - sum(EJi * (hi @ ψ) for EJi, hi in zip(self.EJ,self.HJJ)) \
            + sum((2*g[i,j]) * (N[i] @ (N[j] @ ψ))
                       for i in range(self.nqubits)
                       for j in range(i)
                       if g[i,j])

    def _matvec(self, A):
        return self.apply(A)

    def _matmat(self, A):
        return self.apply(A)

    def tune(self, EJ=None, g=None):
        """Return a new Transmon with tuned parameters."""
        out = copy.copy(self)
        if EJ is not None:
            out.EJ = EJ * np.ones(self.nqubits)
        if g is not None:
            g = g * np.ones((self.nqubits,self.nqubits))
            out.g = 0.5 * (g + g.T)
        return out

    def qubit_basis(self, which=None):
        """Return the computational basis for the transmons in the limit
        of no coupling.
        
        Arguments:
        ----------
        which -- If None, return all 2**nqubits eigenstates. If it is
                 an index, return the eigenstates for the n-th qubit.
        
        Returns:
        --------
        ψ     -- Matrix with columns for the computational basis states.
        """
        nqubits = self.nqubits
        if which is None:
            basis = 1
            for i in range(nqubits):
                basis = np.kron(basis, self.qubit_basis(i))
        else:
            ti = Transmons(nqubits=1, Ec=self.Ec[which],
                           EJ=self.EJ[which], nmax=self.nmax)
            _, basis = lowest_eigenstates(ti, 2)
        return basis
    
    def frequencies(self, n=1):
        """Return gaps between states 1, 2, ... n and the ground state"""
        λ = lowest_eigenvalues(self, neig=n+1)
        return tuple(λ[1:]-λ[0]) if n > 1 else λ[1]-λ[0]

## Applications

### a) Development of band-like structure

We can plot the change of the spectrum as a function of the external potential. Notice how the sensitivity to the external field $n_g$ vanishes as we increase the ratio $E_J/E_c.$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def test():
    ng = np.linspace(-1., 1., 21)
    fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(10,3))
    for (Ec, s, axi) in [(1.0, 'b--', ax1),
                         (1/10., 'k-', ax2),
                         (1/40., 'g-.', ax3)]:
        λ = np.array([lowest_eigenvalues(Transmons(1, Ec, ng=n, EJ=1.), 3)
                      for n in ng])
        axi.plot(ng, λ[:,0], s, label=f'Ec={Ec}')
        axi.plot(ng, λ[:,1:], s)
        axi.set_xlabel('$n_g$')
        axi.set_ylabel('$E/E_J$')
        axi.set_title(f'$E_C={Ec}E_J$')
    plt.tight_layout()
    plt.show()
    
test()

### b) Comparison with exact solutions 

A shown by [J. Koch et al, PRA 76, 042319 (2007)](https://doi.org/10.1103/PhysRevA.76.042319), the transmon eigenenergies are well approximated by the perturbative formulas
$$E_n = -E_J + \sqrt{8 E_c E_J}\left(n +\frac{1}{2}\right) - \frac{E_c}{12}(6n^2+6n+3).$$

This implies that we also have an estimate of the absolute and relative anharmonicities:
$$\alpha = (E_2-E_1) - (E_1-E_0) = \omega_{12} - \omega_{01} = E_2 - 2E_1 + E_0 = -E_c.$$

$$\alpha_r = \frac{\alpha}{E_{01}} = -\sqrt{\frac{E_c}{8E_J}}.$$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math

def test():
    EJEc = np.linspace(0.1, 140, 31)
    Ec = 1.
    E = np.array([lowest_eigenvalues(Transmons(1, Ec=Ec, EJ=Ec*EJEc, nmax=50), 3)
                  for EJEc in EJEc])
    n = np.arange(3)
    Eth = np.array([-EJ + math.sqrt(8*EJ*Ec)*(0.5+n)-Ec*(6*n**2+6*n+3)/12
                    for EJEc in EJEc
                    for EJ in [Ec*EJEc]])
    
    αr = (E[:,2]+E[:,0]-2*E[:,1])/(E[:,1]-E[:,0])
    αrth = -np.sqrt(1./(8*EJEc))

    fig, (ax1,ax2) = plt.subplots(ncols=2,figsize=(9,3),gridspec_kw={'wspace':0.3})
    ax1.plot(EJEc, E, 'r', label='eigenvalues')
    ax1.plot(EJEc, Eth, 'k--', label='perturb.')
    ax1.set_ylabel('$E$')
    ax1.set_xlabel('$E/E_J$')
    ax2.plot(EJEc, αr, 'r', label='eigenvalues')
    ax2.plot(EJEc, αrth, 'k--', label='perturb.')
    ax2.set_ylabel('$\\alpha_r$')
    ax2.set_xlabel('$E/E_J$')
    ax2.set_xlim([10,140])
    ax2.set_ylim([-0.4,0.0])
    plt.tight_layout()
    plt.show()
    
test()

### c) Fit qubits to experimental parameters

If we know the qubit gap $\omega_{01}$ and the anharmonicity $\alpha,$ we can obtain the parameters of the transmon that reproduces those values.

In [ ]:
# file: seeq/models/cqed.py
import scipy.optimize

def fit_qubit(ω01, α, quiet=True, nmax=16, **kwdargs):
    """Compute a Transmons() object that is fitted to have the given
    gap ω01 and anharmonicity α. Returns a Transmons() object."""
    def budget(x):
        t = Transmons(1, Ec=x[0], EJ=x[1], nmax=nmax, **kwdargs)
        ω01x, ω02x = t.frequencies(2)
        αx = (ω02x - 2 * ω01x)
        return [ω01x - ω01, α - αx]
    
    if ω01 < 0 or α > 0 or abs(α) > abs(ω01):
        raise ValueError(f'Invalid transmon properties ω01={ω01}, α={α}')

    αr = α/ω01
    Ec = np.abs(α)
    EJ = ω01**2/(8*Ec)
    if not quiet:
        print('Estimates:')
        print(f'Ec    = {Ec}')
        print(f'EJ    = {EJ}')
        print(f'EJ/Ec = {1./(8*αr**2)} = {EJ/Ec}')

    x = scipy.optimize.fsolve(budget, [Ec, EJ])
    t = Transmons(1, Ec=x[0], EJ=x[1], nmax=nmax, **kwdargs)
    t.α = α
    t.ωq = ω01
    ω01x, ω02x = t.frequencies(2)
    if not quiet:
        print('Computation:')
        print(f'Ec    = {x[0]}')
        print(f'EJ    = {x[1]}')
        print(f'EJ/Ec = {x[1]/x[0]}')
        print(f'ω01/2π= {ω01x/(2*π)}')
        print(f'α/2π  = {(ω02x - 2 * ω01x)/(2*π)}')
    return t

In [ ]:
aux = fit_qubit(2*np.pi*5, 2*np.pi*-0.3)
print(f'Transmon with ω01=5.0GHz, α=-300MHz\nEJ = {aux.EJ[0]}\nEc = {aux.Ec[0]}\n'
      f'frequencies = {aux.frequencies(2)}')

### d) Tune a transmon's frequency

Tuneable frequency transmons typically have a SQUID that allows us to change the Josephson energy. The following function finds out the new value of that Josephson energy so that the qubit has a new gap $\omega_{01}.$ Note that tuning the $E_J$ may change the anharmonicity.

In [ ]:
# file: seeq/models/cqed.py

def tune_qubit(t, ω01, which=0, quiet=True):
    """Create a new Transmons object where one of the qubits has a new
    gap. This tuning is done by changing the Josephson energy of the qubit.
    
    Input:
    t      -- a Transmons object
    ω01    -- the new gap of one qubit
    which  -- which qubit is tuned (default=0)
    quiet  -- if False, print the output of the computation
    
    Output:
    newt   -- a new Transmons object where the properties of one qubit
              have been tuned.
    """
    def budget(x):
        EJ = t.EJ.copy()
        EJ[which] = abs(x)
        newt = t.tune(EJ=np.abs(x))
        ω01x = newt.frequencies()
        return ω01x - ω01
    
    x0 = ω01**2/(8.*t.Ec)
    if not quiet:
        print('Initial:')
        print(f'Ec        = {t.Ec}')
        print(f'EJ        = {t.EJ}')
        print(f'EJ/Ec     = {t.EJ/t.Ec}')
        print(f'ω01/(2*π) = {t.frequencies()/(2*np.pi)}')

    x = scipy.optimize.root(budget, x0).x
    EJ = t.EJ.copy()
    EJ[which] = x
    newt = t.tune(EJ=EJ)
    if not quiet:
        print('Final:')
        print(f'EJ        = {EJ}')
        print(f'ω01/(2*π) = {newt.frequencies()/(2*np.pi)}')
    return newt

In [ ]:
aux = fit_qubit(2*np.pi*5.0, 2*np.pi*-0.3)
tune_qubit(aux, 2*np.pi*4.9, quiet=False)